In [3]:
import numpy as np
import matplotlib as plt
import cirq

In [3]:
import qiskit
from qiskit import IBMQ

IBMQ.save_account('5a5f578d872f079283abb55bbfe0e4955098df699427d827b5a1ca1c74c0e1de848e7d0832a8d79abeaf53af99f8fcb730c833f45175a2775f3fd099ac2e352a')
qiskit.__qiskit_version__

('0.10.0',
 {'qiskit-terra': '0.10.0',
  'qiskit-aer': '0.3.2',
  'qiskit-ignis': '0.2.0',
  'qiskit-ibmq-provider': '0.3.3',
  'qiskit-aqua': '0.6.1',
  'qiskit': '0.13.0'})

In [4]:
print(cirq.google.Bristlecone)

                                             (0, 5)────(0, 6)
                                             │         │
                                             │         │
                                    (1, 4)───(1, 5)────(1, 6)────(1, 7)
                                    │        │         │         │
                                    │        │         │         │
                           (2, 3)───(2, 4)───(2, 5)────(2, 6)────(2, 7)───(2, 8)
                           │        │        │         │         │        │
                           │        │        │         │         │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)────(3, 6)────(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │         │         │        │        │
                  │        │        │        │         │         │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)────(4, 6)────(4, 7)───(4, 8)───(4, 9)───(4, 10)
         │        │      

In [7]:
class XZOptimizer(cirq.PointOptimizer):
    """Replaces an X followed by a Z with a Y."""
    
    # The circuit to improve
    # The index of the moment with the operation to focus on
    # The operation to focus improvements upon
    def optimization_at(self, circuit, index, op):
        # Is the gate an X gate?
        if isinstance(op, cirq.GateOperation) and (op.gate == cirq.X):
        
            # Finds the index of the next moment that touches the given qubits.
            next_op_index = circuit.next_moment_operating_on(op.qubits, index + 1)
            qubit = op.qubits[0]
            
            # If the next index exists
            if next_op_index is not None:
                # Get the operation at the existing index
                next_op = circuit.operation_at(qubit, next_op_index)
                
                # Is this next_op a Z gate?
                if isinstance(next_op, cirq.GateOperation) and (next_op.gate == cirq.Z):
                    # If yes, replace the X and the Z with a Y
                    new_op = cirq.Y.on(qubit)
            
                    # see https://cirq.readthedocs.io/en/stable/generated/ ...
                    # ... cirq.PointOptimizationSummary.html?highlight=pointoptimizationsummary
                    return cirq.PointOptimizationSummary(
                        clear_span = next_op_index - index + 1, # Range of moments to affect. 
                        clear_qubits = op.qubits, # set of qubits cleared with each affected moment
                        new_operations = [new_op]) # operations to replace

# The optimizer
opt = XZOptimizer()

a = cirq.NamedQubit("a")
b = cirq.NamedQubit("b")

# Unoptimized circuit
circuit = cirq.Circuit.from_ops(cirq.X(a), cirq.Z(a), cirq.CZ(a, b), cirq.X(a))
print('Before\n{}\n'. format(circuit))

# Optimized circuit
opt.optimize_circuit(circuit)
print('After\n{}'.format(circuit))

Before
a: ───X───Z───@───X───
              │
b: ───────────@───────

After
a: ───Y───────@───X───
              │
b: ───────────@───────


Write a PointOptimizer that performs (greedily) the simplification that

a: ───H───@───H───  
          │  
b: ───H───X───H───  

is equal to

a: ───X───  
      │  
b: ───@───

In [45]:
class HHOptimizer(cirq.PointOptimizer):
    """Replaces an H followed by an H with identity."""
    
    # The circuit to improve
    # The index of the moment with the operation to focus on
    # The operation to focus improvements upon
    def optimization_at(self, circuit, index, op):
        # Is the gate an H gate?
        if isinstance(op, cirq.GateOperation) and (op.gate == cirq.H):
        
            # Finds the index of the next moment that touches the given qubits.
            next_op_index = circuit.next_moment_operating_on(op.qubits, index + 1)
            qubit = op.qubits[0]
            
            # If the next index exists
            if next_op_index is not None:
                # Get the operation at the existing index
                next_op = circuit.operation_at(qubit, next_op_index)
                
                # Is this next_op a Z gate?
                if isinstance(next_op, cirq.GateOperation) and (next_op.gate == cirq.H):
                    # see https://cirq.readthedocs.io/en/stable/generated/ ...
                    # ... cirq.PointOptimizationSummary.html?highlight=pointoptimizationsummary
                    return cirq.PointOptimizationSummary(
                        clear_span = next_op_index - index + 1, # Range of moments to affect. 
                        clear_qubits = op.qubits, # set of qubits cleared with each affected moment
                        new_operations = []) # operations to replace

# The optimizer
opt = HHOptimizer()

a = cirq.NamedQubit("a")
b = cirq.NamedQubit("b")

# Unoptimized circuit
circuit = cirq.Circuit.from_ops(cirq.H(a), cirq.H(a), cirq.CNOT(a, b), cirq.H(a), cirq.H(b), cirq.H(a),
                                cirq.CNOT(b, a), cirq.H(b), cirq.H(b))
print('Before\n{}\n'. format(circuit))

# Optimized circuit
opt.optimize_circuit(circuit)
cirq.DropEmptyMoments.optimize_circuit(circuit, circuit)
print('After\n{}'.format(circuit))

Before
a: ───H───H───@───H───H───X───────────
              │           │
b: ───────────X───H───────@───H───H───

After
a: ───@───────X───
      │       │
b: ───X───H───@───


In [46]:
class flippedCNOTOptimizer(cirq.PointOptimizer):
    """Replaces an X followed by a Z with a Y."""
    
    # The circuit to improve
    # The index of the moment with the operation to focus on
    # The operation to focus improvements upon
    def optimization_at(self, circuit, index, op):
        # Is the gate a CNOT gate?
        if isinstance(op, cirq.GateOperation) and (op.gate == cirq.CNOT):
            
            # Finds the index of the next moment that touches the given qubits.
            next_op_index = circuit.next_moment_operating_on(op.qubits, index + 1)
            qubit0 = op.qubits[0]
            qubit1 = op.qubits[1]
            
            # If the next index exists
            if next_op_index is not None:
            
                # Get the operation at the existing index
                next_op0 = circuit.operation_at(qubit0, next_op_index)
                next_op1 = circuit.operation_at(qubit1, next_op_index)
                
                # Are next_op0 & next_op1 H gates?
                if isinstance(next_op0, cirq.GateOperation) and (next_op0.gate == cirq.H)
                and (next_op1.gate == cirq.H):
                    new_op = [cirq.H(qubit0), cirq.H(qubit1), cirq.CNOT.on(qubit1, qubit0)]
                
                    # see https://cirq.readthedocs.io/en/stable/generated/ ...
                    # ... cirq.PointOptimizationSummary.html?highlight=pointoptimizationsummary
                    return cirq.PointOptimizationSummary(
                        clear_span = next_op_index - index + 1, # Range of moments to affect
                        clear_qubits = op.qubits, # set of qubits cleared with each affected moment
                        new_operations = [new_op])# operations to replace

# The optimizer
optCNOT = flippedCNOTOptimizer()
optHH = HHOptimizer()

a = cirq.NamedQubit("a")
b = cirq.NamedQubit("b")

# Unoptimized circuit
circuit = cirq.Circuit.from_ops(cirq.H(a), cirq.H(b), cirq.CNOT(a, b), cirq.H(a), cirq.H(b))
print('Before\n{}\n'. format(circuit))

# Optimized circuit
optCNOT.optimize_circuit(circuit)
optHH.optimize_circuit(circuit)
cirq.DropEmptyMoments.optimize_circuit(circuit, circuit)

print('After\n{}'.format(circuit))

Before
a: ───H───@───H───
          │
b: ───H───X───H───

After
a: ───X───
      │
b: ───@───
